In [3]:
import sys  
import pandas as pd
from datetime import datetime

In [17]:
matrix_ideam_path = r'../data/variables/data-ideam-month-fill.zip'
matrix_fcl_path = r'../data/variables/FCL316Basins_month_fill.zip'
matrix_output = r'../data/variables/data-month-fill.csv'

In [5]:
dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d')

In [6]:
df_ideam = pd.read_csv(matrix_ideam_path, parse_dates = ['fecha'], date_parser = dateparse)

In [7]:
df_fcl = pd.read_csv(matrix_fcl_path)

In [8]:
print(df_ideam.shape)
print(df_fcl.shape)

(57600, 13)
(75840, 18)


In [9]:
df = df_ideam.merge(df_fcl, left_on=['cuenca','anio','mes'], right_on=['basin_id','year','month'], how = 'inner' )

In [10]:
print(df.shape)

(57600, 31)


In [11]:
316*12*20

75840

In [12]:
df.drop(columns = [#'fecha', 'cuenca', 
            #'PTPM_CON_max', 'PTPM_CON_median', 'PTPM_CON_min','Q_MEDIA_D_median', 'Q_MN_D_min', 'Q_MX_D_max', 'TMN_CON_min', 'TMX_CON_max', 
           #'elevacion', 'anio', 'mes', 
            'basin_id', 'cod_ah', 'nom_ah', 'cod_szh', 'nom_szh', 'cod_zh', 'nom_zh', 'ruleid',
            'shape_area', 
            #'shape_leng','basin_area', 
            'year', 'month', 'type'
           #'loss_cover_histo', 'loss_cover_fcl', 'loss_cover_fcl_perc','loss_cover_perc_acum'
        ], inplace = True)

In [13]:
df.rename(columns = {'shape_leng': 'basin_perimeter'}, inplace = True)

In [14]:
df.head()

,fecha,cuenca,PTPM_CON_max,PTPM_CON_median,PTPM_CON_min,Q_MEDIA_D_median,Q_MN_D_min,Q_MX_D_max,TMN_CON_min,TMX_CON_max,elevacion,anio,mes,basin_perimeter,basin_area,loss_cover_histo,loss_cover_fcl,loss_cover_fcl_perc,loss_cover_perc_acum
0,2000-01-01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,1,0.358011,2683.774,NaN,NaN,NaN,NaN
1,2000-02-01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,2,0.358011,2683.774,NaN,NaN,NaN,NaN
2,2000-03-01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,3,0.358011,2683.774,NaN,NaN,NaN,NaN
3,2000-04-01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,4,0.358011,2683.774,NaN,NaN,NaN,NaN
4,2000-05-01,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2000,5,0.358011,2683.774,NaN,NaN,NaN,NaN


In [18]:
df.to_csv(matrix_output, index = False)